In [22]:
# This code includes ID3 and C4.5 algorithm 
# set the modelName to ID3
modelName = 'ID3' 
nFeature = 3 

In [23]:
featureNames = []

# read in training dataset 
trainingInstances = []
fr = open('train.txt','rb')
line = fr.readline()
arr = line.strip('\r\n').split('\t')
for i in range(nFeature):
    featureNames.append(arr[i+3])
for line in fr:
    arr = line.strip('\r\n').split('\t')
    featureValues = arr[3:nFeature+3]
    label = arr[nFeature+3]
    trainingInstances.append([featureValues,label])
fr.close()

In [24]:
# show the data format that is using later
print 'Training data:'
nTrainingInstance = len(trainingInstances)
for i in range(nTrainingInstance):
    print i+1,trainingInstances[i]


Training data:
1 [['Home', 'Out', 'NBC'], 'Win']
2 [['Away', 'Out', 'ABC'], 'Win']
3 [['Home', 'In', 'NBC'], 'Win']
4 [['Home', 'Out', 'NBC'], 'Win']
5 [['Away', 'In', 'ABC'], 'Lose']
6 [['Home', 'Out', 'NBC'], 'Win']
7 [['Home', 'In', 'NBC'], 'Win']
8 [['Away', 'Out', 'ABC'], 'Win']
9 [['Away', 'Out', 'ABC'], 'Win']
10 [['Home', 'Out', 'NBC'], 'Win']
11 [['Away', 'Out', 'NBC'], 'Win']
12 [['Away', 'In', 'FOX'], 'Lose']
13 [['Away', 'Out', 'ABC'], 'Lose']
14 [['Home', 'Out', 'NBC'], 'Win']
15 [['Home', 'Out', 'NBC'], 'Lose']
16 [['Home', 'Out', 'NBC'], 'Lose']
17 [['Home', 'Out', 'ESPN'], 'Win']
18 [['Away', 'Out', 'ABC'], 'Lose']
19 [['Home', 'In', 'NBC'], 'Lose']
20 [['Home', 'Out', 'NBC'], 'Win']
21 [['Home', 'Out', 'CBS'], 'Lose']
22 [['Home', 'Out', 'NBC'], 'Win']
23 [['Home', 'In', 'NBC'], 'Lose']
24 [['Away', 'In', 'ABC'], 'Lose']


In [25]:
import math

# Entropy Function
def Entropy(ns):
    ret = 0.0
    nTotal = sum(ns)
    for n in ns:
        ret += -1.0*n/nTotal*math.log(1.0*n/nTotal,2)
    return ret


In [26]:
# Information Gain --> ID3 choose attribute with maximum Information Gain for each level
def IG(instanceIDs,featureID):
    label2count = {}
    value2label2count = {}
    for instanceID in instanceIDs:
        featureValues,label = trainingInstances[instanceID]
        if not label in label2count:
            label2count[label] = 0
        label2count[label] += 1
        featureValue = featureValues[featureID]
        if not featureValue in value2label2count:
            value2label2count[featureValue] = {}
        if not label in value2label2count[featureValue]:
            value2label2count[featureValue][label] = 0
        value2label2count[featureValue][label] += 1
    N = len(instanceIDs)
    ns = []
    for [label,count] in label2count.items():
        ns.append(count)
    H_Y = Entropy(ns)
    H_Y_X = 0.0
    for [value,label2count] in value2label2count.items():
        ns = []
        for [label,count] in label2count.items():
            ns.append(count)
        H_Y_X += 1.0*sum(ns)/N*Entropy(ns)
    return H_Y-H_Y_X


In [27]:
# Gain Ratio --> C4.5 choose attribute with maximum Gain Ratio for each level
def GainRatio(instanceIDs,featureID):
    ret = IG(instanceIDs,featureID)
    if ret == 0: return ret
    value2count = {}
    for instanceID in instanceIDs:
        featureValues,label = trainingInstances[instanceID]
        featureValue = featureValues[featureID]
        if not featureValue in value2count:
            value2count[featureValue] = 0
        value2count[featureValue] += 1
    ns = []
    for [value,count] in value2count.items():
        ns.append(count)
    splitInfo = Entropy(ns)
    return ret/splitInfo

In [28]:
# instanceIDs,featureIDs,level,subTree,bestFeatureID,majorityLabel,parentMajorityLabel
def ConstructDecisionTree(tree):

    instanceIDs,featureIDs,level,_,_,_,parentMajorityLabel = tree

    label2count = {}
    for instanceID in instanceIDs:
        label = trainingInstances[instanceID][1]
        if not label in label2count:
            label2count[label] = 0
        label2count[label] += 1
    label_count = sorted(label2count.items(),key=lambda x:-x[1])
    majorityLabel = label_count[0][0]
    if len(label_count) > 1 and label_count[0][1] == label_count[1][1]:
        majorityLabel = parentMajorityLabel
    tree[5] = majorityLabel
    print '',majorityLabel,label_count
    if len(label_count) == 1: return
    if len(featureIDs) == 0: return

    featureID2criterionScore = {}
    for featureID in featureIDs:
        if modelName == 'ID3':
            criterionScore = IG(instanceIDs,featureID)
        elif modelName == 'C4.5':
            criterionScore = GainRatio(instanceIDs,featureID)            
        else:
            criterionScore = IG(instanceIDs,featureID)
        featureID2criterionScore[featureID] = criterionScore
    featureID_criterionScore = sorted(featureID2criterionScore.items(),key=lambda x:-x[1])
    bestFeatureID,bestCriterionScore = featureID_criterionScore[0][0],featureID_criterionScore[0][1]
    if bestCriterionScore == 0.0: return
    tree[4] = bestFeatureID

    subFeatureIDs = set()
    for featureID in featureIDs:
        if featureID == bestFeatureID: continue
        subFeatureIDs.add(featureID)
    level += 1
    value2subInstanceIDs = {}
    for instanceID in instanceIDs:
        value = trainingInstances[instanceID][0][bestFeatureID]
        if not value in value2subInstanceIDs:
            value2subInstanceIDs[value] = set()
        value2subInstanceIDs[value].add(instanceID)
    for [value,subInstanceIDs] in value2subInstanceIDs.items():
        tree[3][value] = [subInstanceIDs,subFeatureIDs,level,{},None,None,majorityLabel]
        print level,featureNames[bestFeatureID],bestCriterionScore,value
        ConstructDecisionTree(tree[3][value])

print 'Construction:'
tree = [set(range(nTrainingInstance)),set(range(nFeature)),0,{},None,None,None]
print 0
ConstructDecisionTree(tree)

Construction:
0
 Win [('Win', 14), ('Lose', 10)]
1 Media 0.189023551977 NBC
 Win [('Win', 10), ('Lose', 4)]
2 Is_Opponent_in_AP25_Preseason 0.0617433579328 In
 Win [('Win', 2), ('Lose', 2)]
2 Is_Opponent_in_AP25_Preseason 0.0617433579328 Out
 Win [('Win', 8), ('Lose', 2)]
3 Is_Home_or_Away 0.0341440390296 Home
 Win [('Win', 7), ('Lose', 2)]
3 Is_Home_or_Away 0.0341440390296 Away
 Win [('Win', 1)]
1 Media 0.189023551977 CBS
 Lose [('Lose', 1)]
1 Media 0.189023551977 ABC
 Lose [('Lose', 4), ('Win', 3)]
2 Is_Opponent_in_AP25_Preseason 0.291691997138 In
 Lose [('Lose', 2)]
2 Is_Opponent_in_AP25_Preseason 0.291691997138 Out
 Win [('Win', 3), ('Lose', 2)]
1 Media 0.189023551977 FOX
 Lose [('Lose', 1)]
1 Media 0.189023551977 ESPN
 Win [('Win', 1)]


In [29]:
# traverse constructed tree in top to bottom format
def Traverse(tree,_tab):
    instanceIDs,featureIDs,level,subTree,bestFeatureID,majorityLabel,parentMajorityLabel = tree
    print _tab,level,'(',majorityLabel,')'
    if tree[4] == None: return
    _tab += '  '
    print _tab,featureNames[bestFeatureID]
    _tab += '  '    
    for [value,branch] in sorted(subTree.items(),key=lambda x:x[0]):
        print _tab,value
        Traverse(branch,_tab)

print 'Traverse:'
Traverse(tree,'')


Traverse:
 0 ( Win )
   Media
     ABC
     1 ( Lose )
       Is_Opponent_in_AP25_Preseason
         In
         2 ( Lose )
         Out
         2 ( Win )
     CBS
     1 ( Lose )
     ESPN
     1 ( Win )
     FOX
     1 ( Lose )
     NBC
     1 ( Win )
       Is_Opponent_in_AP25_Preseason
         In
         2 ( Win )
         Out
         2 ( Win )
           Is_Home_or_Away
             Away
             3 ( Win )
             Home
             3 ( Win )


In [30]:
# Evaluation Process start
def Predict(tree,featureValues):
    instanceIDs,featureIDs,level,subTree,bestFeatureID,majorityLabel,parentMajorityLabel = tree
    if tree[4] == None: return majorityLabel
    value = featureValues[bestFeatureID]
    if not value in subTree: return majorityLabel
    return Predict(subTree[value],featureValues)

testInstances = []
fr = open('test.txt','rb')
line = fr.readline()
for line in fr:
    arr = line.strip('\r\n').split('\t')
    featureValues = arr[3:nFeature+3]
    label = arr[nFeature+3]
    testInstances.append([featureValues,label])
fr.close()

TP,FP,TN,FN = 0,0,0,0

print 'Test data:'
nTestInstance = len(testInstances)
for i in range(nTestInstance):
    featureValues = testInstances[i][0]
    truthLabel = testInstances[i][1]
    predictLabel = Predict(tree,featureValues)
    print i+1+nTrainingInstance,featureValues,'ground-truth:',truthLabel,'predict:',predictLabel
    if truthLabel == 'Win' and predictLabel == 'Win': TP += 1
    if truthLabel == 'Lose' and predictLabel == 'Win': FP += 1
    if truthLabel == 'Win' and predictLabel == 'Lose': FN += 1
    if truthLabel == 'Lose' and predictLabel == 'Lose': TN += 1


Test data:
25 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
26 ['Home', 'In', 'NBC'] ground-truth: Lose predict: Win
27 ['Away', 'Out', 'ESPN'] ground-truth: Win predict: Win
28 ['Away', 'Out', 'FOX'] ground-truth: Win predict: Lose
29 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
30 ['Away', 'Out', 'ABC'] ground-truth: Win predict: Win
31 ['Home', 'In', 'NBC'] ground-truth: Win predict: Win
32 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
33 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
34 ['Away', 'In', 'ABC'] ground-truth: Lose predict: Lose
35 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
36 ['Away', 'In', 'ABC'] ground-truth: Lose predict: Lose


In [31]:
print 'Confusion matrix:'
print '---------------'
print '| TP',TP,'|','FN',FN,'|'
print '| FP',FP,'|','TN',TN,'|'
print '---------------'

accuracy = 1.0*(TP+TN)/nTestInstance
precision = 0.0
if TP+FP > 0: precision = 1.0*TP/(TP+FP)
recall = 0.0
if TP+FN > 0: recall = 1.0*TP/(TP+FN)
f1 = 0.0
if precision > 0.0 and recall > 0.0: f1 = 2*precision*recall/(precision+recall)

print 'Accuracy:',accuracy
print 'Precision:',precision
print 'Recall:',recall
print 'F1:',f1

Confusion matrix:
---------------
| TP 8 | FN 1 |
| FP 1 | TN 2 |
---------------
Accuracy: 0.833333333333
Precision: 0.888888888889
Recall: 0.888888888889
F1: 0.888888888889
